# TREC

In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv("/Users/zhanglikang/Desktop/SC4002_G06/datasets/TREC/train.csv")
df["text"] = df["text"].str.lower()
print(len(df))
print(df.shape)
df.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
5452
(5452, 3)


,label-coarse,label-fine,text
0,0,0,how did serfdom develop in and then leave russ...
1,1,1,what films featured the character popeye doyle ?
2,0,0,how can i find a list of celebrities ' real na...
3,1,2,what fowl grabs the spotlight after the chines...
4,2,3,what is the full form of .com ?


In [2]:
from sklearn.model_selection import train_test_split

# Split the data to create a development set of 500 examples
train_data, dev_data = train_test_split(df, test_size=500, random_state=42)

# Display the size of the training and development sets
len(train_data), len(dev_data)

(4952, 500)

In [3]:
import numpy as np

# Get unique coarse labels
unique_labels = train_data['label-coarse'].unique()

# Randomly select 4 classes
selected_labels = np.random.choice(unique_labels, size=4, replace=False)

# Update the labels in the train and dev sets
train_data = train_data.copy()
dev_data = dev_data.copy()

train_data['new_label'] = train_data['label-coarse'].apply(lambda x: x if x in selected_labels else 'OTHERS')
dev_data['new_label'] = dev_data['label-coarse'].apply(lambda x: x if x in selected_labels else 'OTHERS')

# Display the unique labels in the updated training set
train_data['new_label'].unique()


array([2, 'OTHERS', 5, 4, 3], dtype=object)